In [13]:
from natto import MeCab
import pandas as pd
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)  #DEBUG を INFOに変えるとlogging.debugが出力されなくなる


#授業の使いまわし
def parse2df(text,sysdic="/usr/local/lib/mecab/dic/naist-jdic"):
    """文毎に形態素解析を行い、結果をdataframeに格納して返す
    Args:
      text:形態素解析対象のテキスト
    Returns:
      形態素解析結果を格納したdataframe
      カラムは['文番号','表層', '品詞1','品詞2','品詞3','品詞4','原型','posID']
    """
    #結果格納用の空のDataFrame
    df = pd.DataFrame(index=[], columns=['文番号','表層', '品詞1','品詞2','品詞3','品詞4','原型','posID'])
    
    text = text.split("\n") #改行で分割して配列にする
    while '' in text: #空行は削除
        text.remove('')
    
    parser = MeCab("-d "+sysdic)

    for index,sentence in enumerate(text): 
        logging.debug(sentence)
        nodes = parser.parse(sentence,as_nodes=True)
        for node in nodes:
            if not node.is_eos():
                #品詞情報を分割
                feature = node.feature.split(',')
                #dataframeに追加
                series = pd.Series( [
                    index,          #文番号
                    node.surface,   #表層
                    feature[0],     #品詞1
                    feature[1],     #品詞2     
                    feature[2],     #品詞3
                    feature[3],     #品詞4
                    feature[6],     #原型
                    node.posid      #品詞番号
                ], index=df.columns)
                df = df.append(series, ignore_index = True)
    logging.debug("End : parse2df")  
    df2 = df[df["posID"].isin([36,37,38,40,41,42,43,44,45,46,47,50,51,52,66,67,2,31,10,34])]
    stop_words = ["する"]
    df2 = df2[~df2["原型"].isin(stop_words )]  # ~df.isin(list) で listに含まれないもの となる
    return df2    

In [14]:
from gensim.models import KeyedVectors
model = KeyedVectors.load_word2vec_format('./entity_vector/entity_vector.model.bin', binary=True)

INFO:gensim.models.utils_any2vec:loading projection weights from ./entity_vector/entity_vector.model.bin
INFO:gensim.models.utils_any2vec:loaded (1015474, 200) matrix from ./entity_vector/entity_vector.model.bin


In [15]:
#sample_seiriからcontentが空文字列のものを取り除いたものを読み込む
df = pd.read_csv("./data/sample_seiri2.csv")

In [16]:
from operator import add

doc_vec = []
for i in range(len(df)):
    if i%10 == 0:
        print(i) #進捗確認用
    title_and_content = df.loc[i]["title"] + df.loc[i]["content"]
    words = parse2df(title_and_content)["原型"]
    tmp = [0]*200 #200次元で単語が表現されている
    word_count = 0 #文章中の単語のうちモデルに登録されているものの数
    for word in words:
        if word in model:
            word_count += 1
            tmp = list(map(add, tmp, model[word]))
    doc_vec.append(list(map(lambda x: x/word_count, tmp))) #平均を取る

0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
2

In [21]:
#dfにdoc_vecの項目を追加
df2 = pd.DataFrame({
    "doc_vec": doc_vec
})
df3 = pd.concat([df.loc[:,["title","content","tags"]],df2], axis=1)

#csvに出力
df3.set_index("title").to_csv("./data/sample_doc_vec.csv")